# Technical Test Torre
By: Jorge Aurelio Valdez Osorio

jorge.valdez.osorio@gmail.com

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re

## Initial data exploration and data cleaning

In [2]:
# Add the data in a data frame
data = pd.read_csv('reed_uk.csv')

First of all we are going to see what data do we have.

In [3]:
data.head()

,category,city,company_name,geo,job_board,job_description,job_requirements,job_title,job_type,post_date,salary_offered,state
0,catering jobs,Chudleigh,Haulfryn Group,uk,reed,Apply now New opportunity not to be missed! H...,NaN,Commis Chef,"Permanent, full-time",3/12/2018,"£18,323 per annum",Devon
1,law jobs,Swindon,Reed,uk,reed,Apply now This role requires an experienced s...,NaN,Corporate Legal Secretary,"Permanent, full-time",3/8/2018,"£18,000 - £22,000 per annum",Wiltshire
2,factory jobs,Norfolk,Swanstaff Recruitment Ltd,uk,reed,Apply now Job Title: Warehouse personLocation...,NaN,Warehouse Person,"Permanent, full-time",2/27/2018,Salary not specified,East Anglia
3,law jobs,Hurstpierpoint,Castles Solicitors,uk,reed,Apply now Castles Solicitors are looking for ...,Required skills Admin Case Management Corresp...,Legal Secretary / Admin Assistant,"Permanent, full-time or part-time",2/21/2018,"£15,000 - £17,000 per annum, pro-rata",West Sussex
4,factory jobs,Hertfordshire,Marketplace Group,uk,reed,Apply now International Pharmaceutical Manufa...,NaN,Category Planner - Pharmaceutical,"Permanent, full-time",3/1/2018,"£37,000 - £45,000 per annum",South East England


As a first galnce we can define each field like this:
* category: (str) a category that describes the area of business of the job.
* city: (str) the city where the job is located.
* company_name: (str) company listing the job..
* geo: (str) conuntry where te job is listed
* job_board: (str) ¿?.
* job_description: (str) text descrbing the job listing.
* job_requirements: (str) text thas display the requiremns that an applicant must have to apply for the job.
* job_title: (str) title for the job.
* job_type: (str) This seems to describe two characterístics of the job, the *contract_type* (permanent, temporal, contract, etc) and *daily_work* (full-time or part-time).
* post_date: (date: 'mm/dd/aaaa') publication date of the job listing.
* salary_offered: (str) How much the company is offering to pay for the job.
* state: (str) state where the job is located

Now we see a little description of th data

In [4]:
data.describe()

,category,city,company_name,geo,job_board,job_description,job_requirements,job_title,job_type,post_date,salary_offered,state
count,50000,50000,50000,50000,50000,50000,20548,50000,50000,50000,50000,49980
unique,37,2918,5166,1,1,42057,14887,29155,9,66,7345,167
top,health jobs,London,Hays Specialist Recruitment Limited,uk,reed,Apply on employer's website Add an annual tur...,Required skills Recruitment,Administrator,"Permanent, full-time",3/7/2018,Salary negotiable,London
freq,1930,4349,1830,50000,50000,85,123,162,36864,8472,4539,5900


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   category          50000 non-null  object
 1   city              50000 non-null  object
 2   company_name      50000 non-null  object
 3   geo               50000 non-null  object
 4   job_board         50000 non-null  object
 5   job_description   50000 non-null  object
 6   job_requirements  20548 non-null  object
 7   job_title         50000 non-null  object
 8   job_type          50000 non-null  object
 9   post_date         50000 non-null  object
 10  salary_offered    50000 non-null  object
 11  state             49980 non-null  object
dtypes: object(12)
memory usage: 4.6+ MB


### Data cleaning

### Drop extra columns
With the first table can conclude that the fields *geo* and *job_board* have only one value, thay are note relevant within this data so we drop them.
With the second we see we are having some null values, specially in job requiremnts wher we have more than half the registers as null, also in state we have 20 missing values.

Because we only are missing 20 values and this is nos a significant number of registers we are going to drop de rows with missing state.

In [6]:
data = data.drop(columns=['geo', 'job_board'])
data = data[np.logical_not(data['state'].isnull())].reset_index(drop=True)
data.head()

,category,city,company_name,job_description,job_requirements,job_title,job_type,post_date,salary_offered,state
0,catering jobs,Chudleigh,Haulfryn Group,Apply now New opportunity not to be missed! H...,NaN,Commis Chef,"Permanent, full-time",3/12/2018,"£18,323 per annum",Devon
1,law jobs,Swindon,Reed,Apply now This role requires an experienced s...,NaN,Corporate Legal Secretary,"Permanent, full-time",3/8/2018,"£18,000 - £22,000 per annum",Wiltshire
2,factory jobs,Norfolk,Swanstaff Recruitment Ltd,Apply now Job Title: Warehouse personLocation...,NaN,Warehouse Person,"Permanent, full-time",2/27/2018,Salary not specified,East Anglia
3,law jobs,Hurstpierpoint,Castles Solicitors,Apply now Castles Solicitors are looking for ...,Required skills Admin Case Management Corresp...,Legal Secretary / Admin Assistant,"Permanent, full-time or part-time",2/21/2018,"£15,000 - £17,000 per annum, pro-rata",West Sussex
4,factory jobs,Hertfordshire,Marketplace Group,Apply now International Pharmaceutical Manufa...,NaN,Category Planner - Pharmaceutical,"Permanent, full-time",3/1/2018,"£37,000 - £45,000 per annum",South East England


In [7]:
data.describe()

,category,city,company_name,job_description,job_requirements,job_title,job_type,post_date,salary_offered,state
count,49980,49980,49980,49980,20546,49980,49980,49980,49980,49980
unique,37,2909,5165,42038,14885,29139,9,66,7338,167
top,health jobs,London,Hays Specialist Recruitment Limited,Apply on employer's website Add an annual tur...,Required skills Recruitment,Administrator,"Permanent, full-time",3/7/2018,Salary negotiable,London
freq,1930,4349,1830,85,123,162,36846,8465,4534,5900


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49980 entries, 0 to 49979
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   category          49980 non-null  object
 1   city              49980 non-null  object
 2   company_name      49980 non-null  object
 3   job_description   49980 non-null  object
 4   job_requirements  20546 non-null  object
 5   job_title         49980 non-null  object
 6   job_type          49980 non-null  object
 7   post_date         49980 non-null  object
 8   salary_offered    49980 non-null  object
 9   state             49980 non-null  object
dtypes: object(10)
memory usage: 3.8+ MB


#### Cleaning *job_type*
Now we are going to transform *job_type* to divide it in *contract_type* and *daily_work*

In [9]:
data['contract_type'], data['daily_work'] = np.transpose(data['job_type'].map(lambda x: x.split(',')).to_list())
data = data.drop(columns=['job_type'])
data.head()

,category,city,company_name,job_description,job_requirements,job_title,post_date,salary_offered,state,contract_type,daily_work
0,catering jobs,Chudleigh,Haulfryn Group,Apply now New opportunity not to be missed! H...,NaN,Commis Chef,3/12/2018,"£18,323 per annum",Devon,Permanent,full-time
1,law jobs,Swindon,Reed,Apply now This role requires an experienced s...,NaN,Corporate Legal Secretary,3/8/2018,"£18,000 - £22,000 per annum",Wiltshire,Permanent,full-time
2,factory jobs,Norfolk,Swanstaff Recruitment Ltd,Apply now Job Title: Warehouse personLocation...,NaN,Warehouse Person,2/27/2018,Salary not specified,East Anglia,Permanent,full-time
3,law jobs,Hurstpierpoint,Castles Solicitors,Apply now Castles Solicitors are looking for ...,Required skills Admin Case Management Corresp...,Legal Secretary / Admin Assistant,2/21/2018,"£15,000 - £17,000 per annum, pro-rata",West Sussex,Permanent,full-time or part-time
4,factory jobs,Hertfordshire,Marketplace Group,Apply now International Pharmaceutical Manufa...,NaN,Category Planner - Pharmaceutical,3/1/2018,"£37,000 - £45,000 per annum",South East England,Permanent,full-time


Now we change this variables to dummy variables.
First contract_type 

In [10]:
data['contract_type'].drop_duplicates()

0     Permanent
14     Contract
17    Temporary
Name: contract_type, dtype: object

In [11]:
data = pd.concat([data, pd.get_dummies(data['contract_type'], prefix='c_t')], axis=1) 
data = data.drop(columns=['contract_type'])
data.head()

,category,city,company_name,job_description,job_requirements,job_title,post_date,salary_offered,state,daily_work,c_t_Contract,c_t_Permanent,c_t_Temporary
0,catering jobs,Chudleigh,Haulfryn Group,Apply now New opportunity not to be missed! H...,NaN,Commis Chef,3/12/2018,"£18,323 per annum",Devon,full-time,0,1,0
1,law jobs,Swindon,Reed,Apply now This role requires an experienced s...,NaN,Corporate Legal Secretary,3/8/2018,"£18,000 - £22,000 per annum",Wiltshire,full-time,0,1,0
2,factory jobs,Norfolk,Swanstaff Recruitment Ltd,Apply now Job Title: Warehouse personLocation...,NaN,Warehouse Person,2/27/2018,Salary not specified,East Anglia,full-time,0,1,0
3,law jobs,Hurstpierpoint,Castles Solicitors,Apply now Castles Solicitors are looking for ...,Required skills Admin Case Management Corresp...,Legal Secretary / Admin Assistant,2/21/2018,"£15,000 - £17,000 per annum, pro-rata",West Sussex,full-time or part-time,0,1,0
4,factory jobs,Hertfordshire,Marketplace Group,Apply now International Pharmaceutical Manufa...,NaN,Category Planner - Pharmaceutical,3/1/2018,"£37,000 - £45,000 per annum",South East England,full-time,0,1,0


Now *daily_work*

In [12]:
data['daily_work'].drop_duplicates()

0                  full-time
3     full-time or part-time
5                  part-time
Name: daily_work, dtype: object

In [13]:
data['d_w_fulltime'] = data['daily_work'].map(lambda x: 1 if 'full-time' in x else 0)
data['d_w_parttime'] = data['daily_work'].map(lambda x: 1 if 'part-time' in x else 0)
data = data.drop(columns=['daily_work'])
data.head()

,category,city,company_name,job_description,job_requirements,job_title,post_date,salary_offered,state,c_t_Contract,c_t_Permanent,c_t_Temporary,d_w_fulltime,d_w_parttime
0,catering jobs,Chudleigh,Haulfryn Group,Apply now New opportunity not to be missed! H...,NaN,Commis Chef,3/12/2018,"£18,323 per annum",Devon,0,1,0,1,0
1,law jobs,Swindon,Reed,Apply now This role requires an experienced s...,NaN,Corporate Legal Secretary,3/8/2018,"£18,000 - £22,000 per annum",Wiltshire,0,1,0,1,0
2,factory jobs,Norfolk,Swanstaff Recruitment Ltd,Apply now Job Title: Warehouse personLocation...,NaN,Warehouse Person,2/27/2018,Salary not specified,East Anglia,0,1,0,1,0
3,law jobs,Hurstpierpoint,Castles Solicitors,Apply now Castles Solicitors are looking for ...,Required skills Admin Case Management Corresp...,Legal Secretary / Admin Assistant,2/21/2018,"£15,000 - £17,000 per annum, pro-rata",West Sussex,0,1,0,1,1
4,factory jobs,Hertfordshire,Marketplace Group,Apply now International Pharmaceutical Manufa...,NaN,Category Planner - Pharmaceutical,3/1/2018,"£37,000 - £45,000 per annum",South East England,0,1,0,1,0


### Cleaninig salary

First we eliminate trailing spaces.

In [14]:
data['salary_offered'] = data['salary_offered'].map(lambda x: x.strip())

Now we found all the salaries thar are not specified and change them to Null values

In [15]:
not_spec_re = r'[^0-9]*'
data['salary_offered'][data['salary_offered'].map(lambda x: re.fullmatch(not_spec_re, x) is not None)].drop_duplicates()

2       Salary not specified
12        Competitive salary
41         Salary negotiable
2021         Commission only
Name: salary_offered, dtype: object

We are going to see how many salaries we are missing

In [16]:
len(data['salary_offered'].loc[data['salary_offered'].isin(['Salary not specified', 'Competitive salary', 'Salary negotiable', 'Commission only'])])

8874

As there are not so many missing salaries we are goin to drop them.

In [17]:
data['salary_offered'].loc[data['salary_offered'].isin(['Salary not specified', 'Competitive salary', 'Salary negotiable', 'Commission only'])] = np.nan
data.head()

/home/jorge/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,category,city,company_name,job_description,job_requirements,job_title,post_date,salary_offered,state,c_t_Contract,c_t_Permanent,c_t_Temporary,d_w_fulltime,d_w_parttime
0,catering jobs,Chudleigh,Haulfryn Group,Apply now New opportunity not to be missed! H...,NaN,Commis Chef,3/12/2018,"£18,323 per annum",Devon,0,1,0,1,0
1,law jobs,Swindon,Reed,Apply now This role requires an experienced s...,NaN,Corporate Legal Secretary,3/8/2018,"£18,000 - £22,000 per annum",Wiltshire,0,1,0,1,0
2,factory jobs,Norfolk,Swanstaff Recruitment Ltd,Apply now Job Title: Warehouse personLocation...,NaN,Warehouse Person,2/27/2018,NaN,East Anglia,0,1,0,1,0
3,law jobs,Hurstpierpoint,Castles Solicitors,Apply now Castles Solicitors are looking for ...,Required skills Admin Case Management Corresp...,Legal Secretary / Admin Assistant,2/21/2018,"£15,000 - £17,000 per annum, pro-rata",West Sussex,0,1,0,1,1
4,factory jobs,Hertfordshire,Marketplace Group,Apply now International Pharmaceutical Manufa...,NaN,Category Planner - Pharmaceutical,3/1/2018,"£37,000 - £45,000 per annum",South East England,0,1,0,1,0


In [18]:
data = data.dropna(subset=['salary_offered']).reset_index(drop=True)
data.head()

,category,city,company_name,job_description,job_requirements,job_title,post_date,salary_offered,state,c_t_Contract,c_t_Permanent,c_t_Temporary,d_w_fulltime,d_w_parttime
0,catering jobs,Chudleigh,Haulfryn Group,Apply now New opportunity not to be missed! H...,NaN,Commis Chef,3/12/2018,"£18,323 per annum",Devon,0,1,0,1,0
1,law jobs,Swindon,Reed,Apply now This role requires an experienced s...,NaN,Corporate Legal Secretary,3/8/2018,"£18,000 - £22,000 per annum",Wiltshire,0,1,0,1,0
2,law jobs,Hurstpierpoint,Castles Solicitors,Apply now Castles Solicitors are looking for ...,Required skills Admin Case Management Corresp...,Legal Secretary / Admin Assistant,2/21/2018,"£15,000 - £17,000 per annum, pro-rata",West Sussex,0,1,0,1,1
3,factory jobs,Hertfordshire,Marketplace Group,Apply now International Pharmaceutical Manufa...,NaN,Category Planner - Pharmaceutical,3/1/2018,"£37,000 - £45,000 per annum",South East England,0,1,0,1,0
4,retail jobs,Belfast,Vodafone,Apply on employer's website Retail Advisor - ...,NaN,Retail Adviser 20hrs/week - Belfast Donnegal P...,3/12/2018,"£9,100 - £9,152 per annum, OTE",County Antrim,0,1,0,0,1


We eliminate "up to" and "from" in the salary offers for the regex cleaning.

In [19]:
data['salary_offered'] = data['salary_offered'].replace('up to ', '', regex=True).replace('from ', '', regex=True)

We define temporary columns where we save the currency and amount of the salaries

In [20]:
min_sal_re = re.compile(r'^(?P<curr1>[^0-9- ]{1,4})(?P<money1>[0-9,.]*)')
data['temp_min_sal'] = data['salary_offered'].map(
    lambda x: min_sal_re.search(x).group('curr1', 'money1') if min_sal_re.search(x) else 'Error'
)

max_sal_re = re.compile(r'- (?P<curr2>[^0-9- ]{1,4})(?P<money2>[0-9,.]*)')
data['temp_max_sal'] = data['salary_offered'].map(
    lambda x: max_sal_re.search(x).group('curr2', 'money2') if max_sal_re.search(x) else np.nan
)
data.head()

,category,city,company_name,job_description,job_requirements,job_title,post_date,salary_offered,state,c_t_Contract,c_t_Permanent,c_t_Temporary,d_w_fulltime,d_w_parttime,temp_min_sal,temp_max_sal
0,catering jobs,Chudleigh,Haulfryn Group,Apply now New opportunity not to be missed! H...,NaN,Commis Chef,3/12/2018,"£18,323 per annum",Devon,0,1,0,1,0,"(£, 18,323)",NaN
1,law jobs,Swindon,Reed,Apply now This role requires an experienced s...,NaN,Corporate Legal Secretary,3/8/2018,"£18,000 - £22,000 per annum",Wiltshire,0,1,0,1,0,"(£, 18,000)","(£, 22,000)"
2,law jobs,Hurstpierpoint,Castles Solicitors,Apply now Castles Solicitors are looking for ...,Required skills Admin Case Management Corresp...,Legal Secretary / Admin Assistant,2/21/2018,"£15,000 - £17,000 per annum, pro-rata",West Sussex,0,1,0,1,1,"(£, 15,000)","(£, 17,000)"
3,factory jobs,Hertfordshire,Marketplace Group,Apply now International Pharmaceutical Manufa...,NaN,Category Planner - Pharmaceutical,3/1/2018,"£37,000 - £45,000 per annum",South East England,0,1,0,1,0,"(£, 37,000)","(£, 45,000)"
4,retail jobs,Belfast,Vodafone,Apply on employer's website Retail Advisor - ...,NaN,Retail Adviser 20hrs/week - Belfast Donnegal P...,3/12/2018,"£9,100 - £9,152 per annum, OTE",County Antrim,0,1,0,0,1,"(£, 9,100)","(£, 9,152)"


We confirm we got at least al minimum, the offers that only have one value are saven in the minimum.

In [21]:
data['temp_min_sal'][data['temp_min_sal'] == 'Error']

Series([], Name: temp_min_sal, dtype: object)

Confirmation that there are not empty string in minimum salary

In [22]:
temp = data['temp_min_sal'].map(lambda x: x[1])
(temp[temp==''])

Series([], Name: temp_min_sal, dtype: object)

In [23]:
data['curr_salary'] = data['temp_min_sal'].map(lambda x: x[0])
data['min_salary'] = data['temp_min_sal'].map(lambda x: x[1]).replace(',','', regex=True).astype(float)
data['max_salary'] = data.apply(
    lambda x: x['temp_max_sal'][1] if isinstance(x['temp_max_sal'], tuple) else x['min_salary'],
    axis=1
).replace(',','', regex=True).astype(float)
data = data.drop(columns=['temp_min_sal', 'temp_max_sal'])
data.head()

,category,city,company_name,job_description,job_requirements,job_title,post_date,salary_offered,state,c_t_Contract,c_t_Permanent,c_t_Temporary,d_w_fulltime,d_w_parttime,curr_salary,min_salary,max_salary
0,catering jobs,Chudleigh,Haulfryn Group,Apply now New opportunity not to be missed! H...,NaN,Commis Chef,3/12/2018,"£18,323 per annum",Devon,0,1,0,1,0,£,18323.0,18323.0
1,law jobs,Swindon,Reed,Apply now This role requires an experienced s...,NaN,Corporate Legal Secretary,3/8/2018,"£18,000 - £22,000 per annum",Wiltshire,0,1,0,1,0,£,18000.0,22000.0
2,law jobs,Hurstpierpoint,Castles Solicitors,Apply now Castles Solicitors are looking for ...,Required skills Admin Case Management Corresp...,Legal Secretary / Admin Assistant,2/21/2018,"£15,000 - £17,000 per annum, pro-rata",West Sussex,0,1,0,1,1,£,15000.0,17000.0
3,factory jobs,Hertfordshire,Marketplace Group,Apply now International Pharmaceutical Manufa...,NaN,Category Planner - Pharmaceutical,3/1/2018,"£37,000 - £45,000 per annum",South East England,0,1,0,1,0,£,37000.0,45000.0
4,retail jobs,Belfast,Vodafone,Apply on employer's website Retail Advisor - ...,NaN,Retail Adviser 20hrs/week - Belfast Donnegal P...,3/12/2018,"£9,100 - £9,152 per annum, OTE",County Antrim,0,1,0,0,1,£,9100.0,9152.0


Now we define if the salaries are per year, per day or per hour, to standirize per hour.

In [24]:
def salary_period(x):
    if re.search('per annum', x):
        return 'year'
    if re.search('per day', x):
        return 'day'
    if re.search('per hour', x):
        return 'hour'
    return None
data['salary_period'] = data['salary_offered'].map(salary_period)
data.head()

,category,city,company_name,job_description,job_requirements,job_title,post_date,salary_offered,state,c_t_Contract,c_t_Permanent,c_t_Temporary,d_w_fulltime,d_w_parttime,curr_salary,min_salary,max_salary,salary_period
0,catering jobs,Chudleigh,Haulfryn Group,Apply now New opportunity not to be missed! H...,NaN,Commis Chef,3/12/2018,"£18,323 per annum",Devon,0,1,0,1,0,£,18323.0,18323.0,year
1,law jobs,Swindon,Reed,Apply now This role requires an experienced s...,NaN,Corporate Legal Secretary,3/8/2018,"£18,000 - £22,000 per annum",Wiltshire,0,1,0,1,0,£,18000.0,22000.0,year
2,law jobs,Hurstpierpoint,Castles Solicitors,Apply now Castles Solicitors are looking for ...,Required skills Admin Case Management Corresp...,Legal Secretary / Admin Assistant,2/21/2018,"£15,000 - £17,000 per annum, pro-rata",West Sussex,0,1,0,1,1,£,15000.0,17000.0,year
3,factory jobs,Hertfordshire,Marketplace Group,Apply now International Pharmaceutical Manufa...,NaN,Category Planner - Pharmaceutical,3/1/2018,"£37,000 - £45,000 per annum",South East England,0,1,0,1,0,£,37000.0,45000.0,year
4,retail jobs,Belfast,Vodafone,Apply on employer's website Retail Advisor - ...,NaN,Retail Adviser 20hrs/week - Belfast Donnegal P...,3/12/2018,"£9,100 - £9,152 per annum, OTE",County Antrim,0,1,0,0,1,£,9100.0,9152.0,year


Confirm not null values in *salary_period*

In [25]:
len(data['salary_period'][pd.isnull(data['salary_period'])])

0

Finally we are going to define our maximum and minimum salary in pounds per hour.

In [26]:
# currencies exchange_rates
cur_er = {
    '£': 1,
    'USD$':1.38,
    '‚¬':1,
    'AUD$':1.79
}
# time periods
time_pr = {
    'hour': 1,
    'day':1/8,
    'year': 1/(255*8) # 255 working days in the year * hours per working day
}
# based in https://hello-safe.co.uk/business-insurance/tools/working-days-calculator
    

In [27]:
data['max_salary'] = data.apply(
    lambda x: x['max_salary']*cur_er[x['curr_salary']]*time_pr[x['salary_period']],
    axis=1
)
data['min_salary'] = data.apply(
    lambda x: x['min_salary']*cur_er[x['curr_salary']]*time_pr[x['salary_period']],
    axis=1
)
data = data.drop(columns=['curr_salary','salary_period','salary_offered'])
data.head()

,category,city,company_name,job_description,job_requirements,job_title,post_date,state,c_t_Contract,c_t_Permanent,c_t_Temporary,d_w_fulltime,d_w_parttime,min_salary,max_salary
0,catering jobs,Chudleigh,Haulfryn Group,Apply now New opportunity not to be missed! H...,NaN,Commis Chef,3/12/2018,Devon,0,1,0,1,0,8.981863,8.981863
1,law jobs,Swindon,Reed,Apply now This role requires an experienced s...,NaN,Corporate Legal Secretary,3/8/2018,Wiltshire,0,1,0,1,0,8.823529,10.784314
2,law jobs,Hurstpierpoint,Castles Solicitors,Apply now Castles Solicitors are looking for ...,Required skills Admin Case Management Corresp...,Legal Secretary / Admin Assistant,2/21/2018,West Sussex,0,1,0,1,1,7.352941,8.333333
3,factory jobs,Hertfordshire,Marketplace Group,Apply now International Pharmaceutical Manufa...,NaN,Category Planner - Pharmaceutical,3/1/2018,South East England,0,1,0,1,0,18.137255,22.058824
4,retail jobs,Belfast,Vodafone,Apply on employer's website Retail Advisor - ...,NaN,Retail Adviser 20hrs/week - Belfast Donnegal P...,3/12/2018,County Antrim,0,1,0,0,1,4.460784,4.486275


In [ ]:
data.loc[36]

In [ ]:
min_sal_re = re.compile(r'^((?P<curr1>[^0-9- ]{1,4})(?P<money1>[0-9,.]*))')
x = curr_re.search('£22.00 - £26.00 per hour')
y = curr_re.search('‚¬18,000 - ‚¬19,000 per annum, negotiable')
z = curr_re.search('USD$40,000 - USD$45,000 per annum')
a = curr_re.search('£18,323 per annum')
print(type(x))
x.group('curr1', 'money1'), y.group('curr1', 'money1'), z.group('curr1', 'money1'), a.group('curr1', 'money1')

In [32]:
data['job_description'][0]

' Apply now This role requires an experienced secretary with at least 2 years secretarial experience. Legal experience is an advantage but not essential. Possessing excellent IT, administration, diary management and exceptional organisational skills. The candidate must be adaptable and flexible together with a professional, confident manner and strong communication skills. Must have advanced knowledge of word and excel.Responsibilities:â€¢ Legal secretarial to include audio and copy typing services, producing letters and documents through our case management system. â€¢ Opening new matters on the case management system ensuring all KYC requirements are met. â€¢ Raising interim and final invoices.â€¢ Monitor daily emails and post for the Partner, being proactive in responding to any urgent correspondence ensuring the clientâ€™s needs are met. â€¢ Daily printing of all emails and documents relating to client matters and filing in appropriate files on our case management system. â€¢ Daily

In [43]:
data['job_description'][6]

' Apply now Break Press Operator - Permanent, Â£9.25 â€“ Â£10.15 per hour + Overtime Monday â€“ Friday, 7.30am â€“ 5pm (Early finish on Friday) Based on the outskirts of Thetford Rapidly growing manufacturing and engineering company Main Duties: Operate press brake machinery, typically working with mild steel and aluminiumCreate CNC programs in line with customer requirementsEnsure the correct tools are used for each componentCheck completed work for accuracy and qualityKeep working area tidy and free from hazardsCompetent in press break work and tool settingRead and interpret drawings Essential Skills: Previous experience within similar role essential, however full training will be givenOwn vehicle required due to locationProactive approach within the workplaceGood attention to detail is a must If you would like to apply for this role, email your current up to date CV to with a covering letter outlining your suitability for the role, Ensuring that your CV includes examples of the abov